<a href="https://colab.research.google.com/github/jxm020202/Multilingual-Chatbot-WDSM-CUP/blob/main/deberta-v3-extra-small-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
import kagglehub
kagglehub.login()


UnauthenticatedError: User is not authenticated

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
BASE_PATH = kagglehub.competition_download('wsdm-cup-multilingual-chatbot-arena')


print('Data source import complete.')

100%|██████████| 108M/108M [00:00<00:00, 145MB/s]

Extracting files...


Data source import complete.


In [ ]:
import os



import numpy as np
import pandas as pd
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
from transformers import AutoTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel

from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel

In [ ]:
class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 512  # Input sequence length
    epochs = 3 # Training epochs
    batch_size = 16  # Batch size
    scheduler = 'cosine'  # Learning rate scheduler
    label2name = {0: 'model_a', 1: 'model_b'}
    name2label = {v:k for k, v in label2name.items()}
    class_labels = list(label2name.keys())
    class_names = list(label2name.values())

In [ ]:
data = pd.read_parquet(f'{BASE_PATH}/train.parquet')
# Sample 10% of the dataset
data = data.sample(frac=0.1, random_state=42)
data.to_csv('train.csv', index=False, escapechar="\\")

df = pd.read_csv('train.csv')


df["class_name"] = df[["model_a", "model_b"]].idxmax(axis=1)
df["label"] = df.class_name.map(CFG.name2label)
from datasets import Dataset
# Convert the DataFrame to a Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
df.head(1)

,id,prompt,response_a,response_b,winner,model_a,model_b,language,class_name,label
0,98aa6a2c252836e0f5fb4412f56e711b9a4d77df15be08...,c++如何实现快速幂,```cpp\n#include <iostream>\n\nusing namespace...,在C++中，实现快速幂（也称为快速幂取模）可以通过二分法来优化幂运算的效率，尤其是在处理大数...,model_b,gemma-2-27b-it,grok-2-2024-08-13,Chinese,model_b,1


In [ ]:

#tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
tokenizer = GemmaTokenizerFast.from_pretrained("unsloth/gemma-2-9b-it-bnb-4bit")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    # Combine the prompt with each response to form the options
    first_sentences = [[prompt] * 2 for prompt in examples['prompt']]
    second_sentences = [
        [examples['response_a'][i], examples['response_b'][i]] for i in range(len(examples['prompt']))
    ]

    # Flatten the lists to prepare for tokenization
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize the combined sentences
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)

    # Un-flatten the tokenized inputs to match the original structure
    return {k: [v[i:i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}


In [ ]:
# Assuming 'dataset' is your Hugging Face Dataset object
tokenized_dataset = dataset.map(preprocess_function, batched=True)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Access the train and validation datasets
train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

Map:   0%|          | 0/4844 [00:00<?, ? examples/s]

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

#model = AutoModelForMultipleChoice.from_pretrained("google-bert/bert-base-uncased")
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=2,  # directly load 2 classes from start
    torch_dtype=torch.float16,
    device_map="auto",
)


model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="output",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    logging_steps=5,
    save_steps=50
    warmup_steps=50,

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.686407,0.554500
2,0.692700,0.668501,0.589595


TrainOutput(global_step=606, training_loss=0.6904062453669684, metrics={'train_runtime': 2518.9954, 'train_samples_per_second': 3.846, 'train_steps_per_second': 0.241, 'total_flos': 5097994035412992.0, 'train_loss': 0.6904062453669684, 'epoch': 2.0})